In [ ]:
import base64
import os
import pandas as pd
from io import BytesIO
from flask import Flask, request, url_for
import pickle
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
# from PIL import Image
import numpy as np
from sklearn.preprocessing import StandardScaler

import warnings
app = Flask(__name__)

# Set Upload folder in Flask incase if the uploaded file is saved in localhost 
UPLOAD_FOLDER = os.getcwd()+'/static/files'
app.config['UPLOAD_FOLDER'] =  UPLOAD_FOLDER

def load_model(file_name='BreastCancer_SVM_trained_model_acc99.pkl'):
    model,scaler=pickle.load(open(file_name,'rb'))
    return model,scaler
model,scaler=load_model()
# img = Image.open('numberofB-M.png')
# img_data = BytesIO()
# img.save(img_data, "png")
# #Then encode the saved image file.
# encoded_img_data = base64.b64encode(img_data.getvalue()).decode('utf-8')
# image_base64 = base64.b64encode(img.de).decode('utf-8')


def load_pkl_data(file):
    data= pickle.load(open(file.filename,'rb'))
    return data
# to predict new dataset, the dataset has to have 30 features (columns).
# the function accpets 2 different files .csv and .pkl
def prediction(X_test):
    col_size=0
    # check the type of object if its list
    if(isinstance(X_test,list)):
        col_size=len(X_test[0][:])
        row_size=len(X_test[:])
    # check the type of object if its numpy
    elif(isinstance(X_test,np.ndarray)):
        col_size=X_test.shape[1]
        row_size=X_test.shape[0]
    # check the type of object if its DataFrame
    elif(isinstance(X_test,pd.DataFrame)):
        col_size=X_test.shape[1]
        row_size=X_test.shape[0]
    # check th number of columns in new data if they match with model features
    if(col_size==model.n_features_in_):
        with warnings.catch_warnings():
            scaler = StandardScaler().fit(X_test)
            warnings.simplefilter("ignore")
            X_test_scaled = scaler.transform(X_test)
        predictions = model.predict(X_test_scaled)
        # convert prediction value from int [0,1] to [Malignant,Benign]
        diagnosis_pred = ['Malignant' if x=='1' else 'Benign' for x in predictions]
        
        return predictions,diagnosis_pred,col_size,row_size
    else:
        return f'The number of columns: {col_size} are not match with model features:',f'{model.n_features_in_}. <br>data size:{row_size}'
        
@app.route('/')
def home():
    
    return f'''
        <html>
        <head>
            <title>Breast Cancer Predictor</title>
        </head>
        <body style="background-color:AntiqueWhite;">
            <h1>Breast Cancer Predictor</h1>
            <!--
            <img src="data:image/jpeg;base64,{{image_base64}}" alt="Uploaded Image" width="300">
            -->
            <form action="/predict" method="post" enctype="multipart/form-data">
            <label>Spreadsheet: [Upload .CSV or .PKL file]</label><br>
                <input type="file" name="file" accept=".csv,.pkl" required>
                <input type="submit" value="Predict">
            </form>
        </body>
        </html>
    '''

@app.route('/predict', methods=['POST'])
def predict():
    found= True
    file = request.files['file']
    print(f'file:{file}\n\n')
    print(f'file.filename: {file.filename}')
    # if file.rsplit('.', 1)[1].lower()
    print(file.filename.rsplit('.', 1)[1].lower())
    # data = pd.read_csv(file)
    # print(f'data:{data}')
    # Read the image data
    # file_data = file.read()
    # print(file_data)
    if file.filename != '':
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        print(f'file_path: {file_path}')
        if (file.filename.rsplit('.', 1)[1].lower() =='pkl'):
            data = load_pkl_data(file)
            data = pd.DataFrame(data)
            print(data.head())
        elif (file.filename.rsplit('.', 1)[1].lower() =='csv'): 
            data = pd.read_csv(file,index_col=0)
            print(data.head())
        new_pred,diagnosis_pred,col_size,row_size=prediction(data)
        diagnosis_pred1=pd.DataFrame(diagnosis_pred)
        diagnosis_pred1.columns=['Prediction']
        
    else:
        found=False
      # save file in local host at the file_path
        # file.save(file_path)
      # save the file
    return f'''
        <html>
        <head>
            <title>Result</title>
            <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              text-align: left;
              padding: 8px;
            }}
            tr:nth-child(even) {{
              background-color: #D6EEEE;
            }}
            </style>
        </head>
        <body style="background-color:AntiqueWhite">
            <h1>Data Information:</h1>
            <p>{data.to_html() if found else 'The type of file is not supported!'}</p>
            <br>
            <!--
            <p>{new_pred}</p
            <p>{diagnosis_pred}</p>
            -->
            <h1>Predict Results</h1>
            
            <p>{diagnosis_pred1.to_html()}</p>
        </body>
        </html>
    '''

if __name__ == '__main__':
    app.run(debug=False,port=5005)
    # app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


/Users/asrar/anaconda3/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/asrar/anaconda3/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
 * Running on http://127.0.0.1:5005
Press CTRL+C to quit
127.0.0.1 - - [21/Jul/2023 22:16:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2023 22:17:06] "POST /predict HTTP/1.1" 200 -


file:<FileStorage: 'UploadDataFlask.csv' ('text/csv')>


file.filename: UploadDataFlask.csv
csv
file_path: /Users/asrar/Desktop/FinalProject2/static/files/UploadDataFlask.csv
       0      1       2       3        4        5       6        7       8  \
0  10.86  21.48   68.51   360.5  0.07431  0.04227  0.0000  0.00000  0.1661   
1  14.95  18.77   97.84   689.5  0.08138  0.11670  0.0905  0.03562  0.1744   
2  20.29  14.34  135.10  1297.0  0.10030  0.13280  0.1980  0.10430  0.1809   
3  18.05  16.15  120.20  1006.0  0.10650  0.21460  0.1684  0.10800  0.2152   
4  23.51  24.27  155.10  1747.0  0.10690  0.12830  0.2308  0.14100  0.1797   

         9  ...     20     21      22      23      24       25      26  \
0  0.05948  ...  11.66  24.77   74.08   412.3  0.1001  0.07348  0.0000   
1  0.06493  ...  16.25  25.47  107.10   809.7  0.0997  0.25210  0.2500   
2  0.05883  ...  22.54  16.67  152.20  1575.0  0.1374  0.20500  0.4000   
3  0.06673  ...  22.39  18.91  150.10  1610.0  0.1478  0.563

In [ ]:
os.getcwd()+'/numberofB-M.png'